# 来自Transformers的双向编码器表示（BERT）
## 从上下文无关到上下文敏感
## 从特定于任务到不可知任务
## BERT：把两个最好的结合起来

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 输入表示



In [2]:
'''
输入：两个词元列表（tokens_a和可选的tokens_b）
输出：添加特殊标记后的词元序列和片段标记序列
'''
#@save
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    '''
    1. 构建基础序列
    '<cls>'：在序列开头添加分类标记（Class Token），BERT用它表示整个句子的语义
    tokens_a：第一个句子/段落的词元列表
    '<sep>'：在句子A末尾添加分隔标记（Separator Token），标记句子A结束
    '''
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    '''
    2. 构建片段标记（Segment A）
    为序列中所有属于句子A的词元分配片段ID 0
    len(tokens_a)+2：包含 <cls>、tokens_a 的所有词和<sep>
    结果：[0,0,0,...,0]（长度=len(tokens_a)+2）
    '''
    segments = [0] * (len(tokens_a) + 2)
    # 3. 处理句子B：如果提供了tokens_b
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>'] # 将句子B的词元和末尾的<sep>追加到序列
        segments += [1] * (len(tokens_b) + 1) # 为句子B的词元和最后的<sep>分配片段ID 1
    return tokens, segments

| 组件                      | 作用             | 关键技术            |
| ----------------------- | -------------- | --------------- |
| **`token_embedding`**   | 词元 → 向量        | `nn.Embedding`  |
| **`segment_embedding`** | 句子标记 → 向量      | 区分两个句子          |
| **`pos_embedding`**     | 位置信息           | 可学习的参数          |
| **`blks`**              | Transformer层堆叠 | `num_layers`次循环 |
| **相加操作**                | 融合三种信息         | 逐元素相加           |


**在 blk(X, valid_lens) 内部的作用**

每个 EncoderBlock 都包含多头注意力层，valid_lens 用于：

1. 生成注意力掩码
```Python
# 在 EncoderBlock 内部
def forward(self, X, valid_lens):
    # X 形状: (batch_size, seq_len, num_hiddens)
    # valid_lens 形状: (batch_size,)
    
    # 根据 valid_lens 生成掩码
    # 例如 valid_lens=[2, 3]，seq_len=4
    # 掩码: [[0, 0, 1, 1], [0, 0, 0, 1]]
    # (0=有效, 1=填充)
    
    # 在注意力计算中，填充位置的注意力分数设为 -inf
    # Softmax 后变为 0，模型不关注
    attn_output = self.attention(X, X, X, valid_lens)
```

2. 具体实现细节
```Python
# 在 MultiHeadAttention 中
def sequence_mask(self, valid_lens):
    # valid_lens: [2, 3]
    # 返回 shape: (batch_size, seq_len)
    # [[False, False,  True,  True],   # 第1个样本：后2个位置是填充
    #  [False, False, False,  True]]   # 第2个样本：最后1个位置是填充
    
    # 扩展为注意力分数形状 (batch_size, num_heads, seq_len, seq_len)
    mask = self.expand_mask(mask)
    return mask
```

| 参数                          | 切片对象             | 作用          | 结果                                      |
| --------------------------- | ---------------- | ----------- | --------------------------------------- |
|  参数1 `:`            | 批量维度   | 选取所有（大小为1）  | 保持 `(1, ...)`                           |
|  参数2 `:X.shape[1]`  | 序列长度维度 | 动态截取到当前序列长度 | 从 `(..., 1000, ...)` → `(..., 50, ...)` |
|  参数3 `:`           | 嵌入维度   | 选取全部        | 保持 `(..., ..., 768)`                    |


In [3]:
#@save
class BERTEncoder(nn.Module):
    '''
    vocab_size：词汇表大小（如30000）
    num_hiddens：隐藏层维度（如768，与词向量维度一致）
    norm_shape：LayerNorm的形状（通常是num_hiddens）
    ffn_num_input：前馈网络输入维度（通常等于num_hiddens）
    ffn_num_hiddens：前馈网络隐藏层维度（如3072）
    num_heads：多头注意力头数（如12）
    num_layers：Transformer层数（如12）
    dropout：Dropout概率（如0.1）
    max_len：最大序列长度（位置嵌入的预分配大小）
    key_size/query_size/value_size：注意力机制的维度（通常等于num_hiddens）
    '''
    """BERT编码器"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        # 1. 三种嵌入层：词元嵌入、位置嵌入、片段嵌入
        # 词元嵌入，将词元ID转换为向量（vocab_size→num_hiddens）
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        # 片段嵌入，区分两个句子（0或1，2种可能）
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        '''
        2. Transformer编码器块堆叠
        循环创建num_layers个EncoderBlock（Transformer层）
        每个块包含：多头自注意力+前馈网络+残差连接+LayerNorm
        True：表示使用残差连接
        '''
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}", d2l.EncoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape,
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        '''
        位置嵌入：
        Position Embedding：BERT使用可学习的位置嵌入
        torch.randn(1,max_len,num_hiddens)：随机初始化，形状为(1,max_len,num_hiddens)
        nn.Parameter：注册为模型参数，参与训练
        维度1：批量维度（这里固定为1，因为位置嵌入是共享的）
        维度2：序列长度维度（最大长度max_len）
        维度3：嵌入维度（num_hiddens）
        '''
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))
    '''
    tokens：词元ID序列，形状(batch_size,seq_len)
    segments：段标记序列，形状(batch_size,seq_len)（0或1）
    valid_lens：有效长度（用于掩码填充部分），形状(batch_size,)
    '''
    def forward(self, tokens, segments, valid_lens):
        # 在以下代码段中，X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        '''
        第1步：嵌入相加
        Token Embedding：将词元ID转为向量
        Segment Embedding：将段标记ID转为向量
        逐元素相加：X形状保持(batch_size,seq_len,num_hiddens)
        '''
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        '''
        第2步：加上位置嵌入
        .data：获取参数值（避免梯度追踪）
        [:,:X.shape[1],:]：截取前seq_len个位置（因为pos_embedding预分配了max_len）
        逐元素相加：加入位置信息
        '''
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        '''
        第3步：逐个通过Transformer层
        循环通过num_layers个编码器块
        每个块执行：自注意力→前馈→残差连接
        valid_lens：用于创建掩码，忽略填充部分，真实词元数量（即非填充部分的长度）
        '''
        for blk in self.blks:
            X = blk(X, valid_lens)
        '''
        形状：(batch_size,seq_len,num_hiddens)
        含义：每个位置的上下文相关表示（contextualized representation）
        '''
        return X

假设词表大小为10000，为了演示`BERTEncoder`的前向推断，让我们创建一个实例并初始化它的参数。


1. 初始化嵌入层
```Python
# 在 __init__ 中自动创建
self.token_embedding = nn.Embedding(10000, 768)      # 词元嵌入
self.segment_embedding = nn.Embedding(2, 768)        # 片段嵌入（0或1）
self.pos_embedding = nn.Parameter(torch.randn(1, 1000, 768))  # 位置嵌入（随机初始化）
```
2. 堆叠2个Transformer层
```Python
# 在 __init__ 的循环中创建
self.blks = nn.Sequential()
for i in range(2):  # num_layers = 2
    self.blks.add_module(f"{i}", d2l.EncoderBlock(...))
    # 第0层: EncoderBlock(0)
    # 第1层: EncoderBlock(1)
```
每个 EncoderBlock 包含：
- 多头自注意力（4个头）
- 前馈网络（768 → 1024 → 768）
- 残差连接和LayerNorm
3. 总参数量估算
```Python
# 简化的参数计算（忽略偏置）
token_emb = 10000 * 768 ≈ 7.7M
segment_emb = 2 * 768 ≈ 1.5K
pos_emb = 1000 * 768 ≈ 768K

# 每个Transformer层
attention = 4 * (768 * 768) * 3 ≈ 7.1M  # Q,K,V投影
ffn = 768 * 1024 + 1024 * 768 ≈ 1.6M * 2 ≈ 3.2M

# 2层总计
total ≈ 7.7M + 0.8M + (7.1M + 3.2M) * 2 ≈ 7.7M + 0.8M + 20.6M ≈ **29M参数**
（实际BERT-base有1.1亿参数）
```
**得到的 encoder 对象**
```Python
print(encoder)
# 输出：
BERTEncoder(
  (token_embedding): Embedding(10000, 768)
  (segment_embedding): Embedding(2, 768)
  (blks): Sequential(
    (0): EncoderBlock(...)
    (1): EncoderBlock(...)
  )
  (pos_embedding): Parameter containing: [torch.FloatTensor of size 1x1000x768]
)
```
**可执行的操作**
```Python
# 准备输入
tokens = torch.randint(0, 10000, (32, 10))      # 批量32，长度10
segments = torch.zeros(32, 10, dtype=torch.long) # 全0（单句）
valid_lens = torch.tensor([10] * 32)            # 有效长度10

# 前向传播
output = encoder(tokens, segments, valid_lens)
print(output.shape)  # → torch.Size([32, 10, 768])
```

```python
输入：
tokens:   (2, 8)      [[123, 4567, ..., 567],
                       [4321, 987, ..., 7654]]

segments: (2, 8)      [[0,0,0,0,1,1,1,1],
                       [0,0,0,1,1,1,1,1]]

↓ Token Embedding
X_token:  (2, 8, 768) 每个词元ID转为768维向量

↓ Segment Embedding
X_seg:    (2, 8, 768)  每个片段标记转为768维向量

↓ Position Embedding（截取）
X_pos:    (2, 8, 768)  前8个位置的嵌入，广播后相加

↓ Transformer Block × 2
encoded_X: (2, 8, 768)  每个词元的上下文表示
```

In [4]:
'''
vocab_size=10000：词汇表大小（实际BERT通常是30,000-50,000，这里简化为10,000）
num_hiddens=768：隐藏层维度=词向量维度（与BERT-base一致）
ffn_num_hiddens=1024：前馈网络隐藏层维度（实际BERT-base是3072，这里缩减）
num_heads=4：多头注意力头数（实际BERT-base是12，这里减半）
'''
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
'''
norm_shape=[768]：LayerNorm的归一化形状（对最后一个维度768归一化）
ffn_num_input=768：前馈网络输入维度（等于num_hiddens）
num_layers=2：Transformer层数（实际BERT-base是12，这里极度简化）
dropout=0.2：Dropout比率（实际BERT常用0.1）
'''
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input,
                      ffn_num_hiddens, num_heads, num_layers, dropout)

我们将`tokens`定义为长度为8的2个输入序列，其中每个词元是词表的索引。使用输入`tokens`的`BERTEncoder`的前向推断返回编码结果，其中每个词元由向量表示，其长度由超参数`num_hiddens`定义。此超参数通常称为Transformer编码器的*隐藏大小*（隐藏单元数）。


In [5]:
'''
第1行：生成随机词元序列
torch.randint(low,high,shape)：生成随机整数张量
0,vocab_size：词元ID范围是[0,10000)（因为我们定义vocab_size=10000）
(2,8)：批量大小为2，每个样本长度为8
结果：形状(2,8)的随机词元ID矩阵，模拟两个长度为8的句子
'''
tokens = torch.randint(0, vocab_size, (2, 8))
'''
第2行：定义片段标记
作用：标记每个词元属于句子A（0）还是句子B（1）
第一个样本 [0,0,0,0,1,1,1,1]：
    前4个词元（索引0-3）属于句子A
    后4个词元（索引4-7）属于句子B
第二个样本 [0,0,0,1,1,1,1,1]：
    前3个词元属于句子A
    后5个词元属于句子B
'''
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
'''
第3行：执行前向传播
tokens：词元ID张量（形状(2,8)）
segments：片段标记张量（形状(2,8)）
None valid_lens参数设为None：
    表示不指定有效长度（即所有位置都有效）
    等价于valid_lens=torch.tensor([8,8])
    模型不会对填充位置做特殊处理
内部流程：
    token_embedding：将(2,8)转为(2,8,768)
    segment_embedding：将片段标记(2,8)转为(2,8,768)
    pos_embedding：截取前8个位置(1,8,768)并广播到(2,8,768)
    三者相加得到X（形状(2,8,768)）
    通过2个Transformer编码器块（每层有自注意力+前馈网络）
    返回最终编码表示encoded_X
'''
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

torch.Size([2, 8, 768])

## 预训练任务


### 掩蔽语言模型（Masked Language Modeling）

In [6]:
#@save
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        '''
        初始化MLP预测头
        第1个Linear：768→256（将上下文表示映射到隐藏层）
        ReLU：非线性激活
        LayerNorm：稳定训练
        第2个Linear：256→vocab_size（输出词表上的概率分布）

        '''
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size))

    def forward(self, X, pred_positions):
        '''
        第1步：确定预测位置数量
        pred_positions是一个二维张量，形状为(batch_size,num_pred_positions)
        示例：pred_positions=[[1,3,5],[2,4,6]]→shape=(2,3)→num_pred_positions=3
        这表示每个样本要预测3个位置
        '''
        num_pred_positions = pred_positions.shape[1]
        '''
        第2步：展平位置索引
        作用：将二维张量展平为一维，方便后续索引
        示例：[[1,3,5],[2,4,6]]→[1,3,5,2,4,6]
        形状：(2,3)→(6,)（共6个预测位置）
        '''
        pred_positions = pred_positions.reshape(-1)
        '''
        第3步：创建批次索引
        torch.arange(0,batch_size)：生成[0,1,2,...,batch_size-1]
        torch.repeat_interleave(...,num_pred_positions)：将每个批次索引重复3次
        结果：[0,0,0,1,1,1]，与展平后的pred_positions一一对应
        '''
        batch_size = X.shape[0] # 假设X.shape=(2,8,768)→batch_size=2
        batch_idx = torch.arange(0, batch_size)  # →tensor([0,1])
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions) # →tensor([0,0,0,1,1,1])
        '''
        第4步：提取掩蔽位置的表示（核心）
        这是高级索引 操作，从X中精确提取需要预测的位置：
        假设batch_size=2，num_pred_positions=3，那么batch_idx是np.array（[0,0,0,1,1,1]）
        '''
        masked_X = X[batch_idx, pred_positions]
        '''
        第5步：重塑为(batch_size,num_pred_positions,-1)
        目的：将展平后的张量恢复为三维结构，方便MLP按批次处理。
        (6,768)→(2,3,768)
        -1 自动推断为768（特征维度）
        '''
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        # 
        '''
        第6步：通过MLP预测
        输出含义：每个预测位置在词表上的logits分布
        后续会用softmax和交叉熵计算损失
        输入: (2,3,768)
        经过self.mlp:
            Linear(768→256)+ReLU+LayerNorm+Linear(256→10000)
        输出: (2,3,10000)
        '''
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

为了演示`MaskLM`的前向推断，我们创建了其实例`mlm`并对其进行了初始化。回想一下，来自`BERTEncoder`的正向推断`encoded_X`表示2个BERT输入序列。我们将`mlm_positions`定义为在`encoded_X`的任一输入序列中预测的3个指示。`mlm`的前向推断返回`encoded_X`的所有掩蔽位置`mlm_positions`处的预测结果`mlm_Y_hat`。对于每个预测，结果的大小等于词表的大小。


In [7]:
'''
实例化MLM模型
vocab_size=10000：词汇表大小（根据前文）
num_hiddens=256：MLP隐藏层维度（根据MaskLM定义）
创建的MLM结构：MLP: Linear(768→256)→ReLU→LayerNorm→Linear(256→10000)
'''
mlm = MaskLM(vocab_size, num_hiddens)
'''
定义要预测的位置
这是一个2×3的张量，表示2个样本，每个样本需要预测3个位置：
    第0个样本（第0行）：预测位置 1,5,2
    第1个样本（第1行）：预测位置 6,1,5
注意：位置索引是在原始序列中的位置（0-based）。
'''
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
'''
执行MLM前向传播
encoded_X：来自BERT编码器的输出，假设形状为(2,8,768)（2个样本，8个词元，768维）
mlm_positions：预测位置，形状 (2,3)
内部执行流程（回顾MaskLM.forward）：
    1. num_pred_positions=3
    2. pred_positions展平→[1,5,2,6,1,5]
    3. batch_idx→[0,0,0,1,1,1]
    4. 高级索引提取：masked_X=encoded_X[[0,0,0,1,1,1],[1,5,2,6,1,5]]→形状 (6,768)
    5. 重塑：(6,768)→(2,3,768)
    6. MLP预测：(2,3,768)→(2,3,256)→(2,3,10000)
'''
mlm_Y_hat = mlm(encoded_X, mlm_positions)
'''
查看输出形状
2：批量大小（2个样本）
3：每个样本预测3个位置
10000：每个位置在10000个词上的logits分布
'''
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

| 代码行                                               | 作用   | 输入形状                   | 输出形状              | 关键技术    |
| ------------------------------------------------- | ---- | ---------------------- | ----------------- | ------- |
| `mlm_Y`                                  | 真实标签 | -                      | `(2, 3)`          | 词元索引    |
| `nn.CrossEntropyLoss(reduction='none')` | 损失函数 | -                      | -                 | 逐元素损失   |
| `mlm_Y_hat.reshape(-1, vocab_size)`     | 重塑预测 | `(2, 3, 10000)`        | `(6, 10000)`      | 展平批量和位置 |
| `mlm_Y.reshape(-1)`                     | 重塑标签 | `(2, 3)`               | `(6,)`            | 展平标签    |
| `loss(...)`                             | 计算损失 | `(6, 10000)` vs `(6,)` | `(6,)`            | 交叉熵     |
| `.shape`                                | 查看结果 | -                      | `torch.Size([6])` | 验证维度    |



In [8]:
'''
1. 真实标签
含义：真实的词元ID（目标答案），形状 (2,3)
    第0个样本的3个预测位置的真实词分别是索引7,8,9
    第1个样本的3个预测位置的真实词分别是索引10,20,30
'''
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
'''
2. 定义损失函数
CrossEntropyLoss：交叉熵损失，用于分类任务
reduction='none'：返回 每个样本的独立损失，不聚合（不sum/mean）
输出形状：将为每个预测位置返回一个标量损失
'''
loss = nn.CrossEntropyLoss(reduction='none')

'''
3. 计算损失
mlm_Y_hat.reshape((-1,vocab_size))
    mlm_Y_hat形状:(2,3,10000)
    reshape后:(6,10000) # -1 自动计算为2*3=6
    变成: 6个预测位置，每个对应10000个词的logits
mlm_Y.reshape(-1)
    mlm_Y形状:(2,3)
    reshape后:(6,) # -1 自动计算为2*3=6
    变成: 6个真实词索引
损失计算过程
    第1个预测位置:mlm_Y_hat[0,0,:] vs mlm_Y[0](7)
    第2个预测位置:mlm_Y_hat[0,1,:] vs mlm_Y[1](8)
    ...
    第6个预测位置:mlm_Y_hat[1,2,:] vs mlm_Y[5](30)
'''
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
'''
4. 查看损失形状
预期输出：torch.Size([6])
为什么是6？ 因为总共2个样本×3个预测位置=6个独立损失
每个元素：对应一个预测位置的交叉熵损失值
'''
mlm_l.shape

torch.Size([6])

### 下一句预测（Next Sentence Prediction）



| 组件                             | 作用         | 输入                        | 输出                     |
| ------------------------------ | ---------- | ------------------------- | ---------------------- |
|  `NextSentencePred`    | NSP任务头     | `<cls>` 表示 `(batch, 768)` | 二分类logits `(batch, 2)` |
|  `encoded_X[:, 0, :]`  | 提取 `<cls>` | `(batch, seq_len, 768)`   | `(batch, 768)`         |
| 训练目标                   | 理解句间关系     | 句对输入                      | 0/1标签                  |


In [9]:
#@save
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""
    '''
    num_inputs：输入特征维度，通常等于num_hiddens（如768），是<cls>词元的表示维度
    nn.Linear(num_inputs,2)：线性层，将768维向量映射到2个输出（二分类：是/否下一句）
    输出含义：
        logit[0]：不是下一句的分数
        logit[1]：是下一句的分数
    '''
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 2)
    '''
    输入X：<cls>词元的编码表示，形状(batch_size,768)
    为什么是<cls>？因为BERT训练时让<cls>聚合整个序列的语义信息，适合分类任务
    返回：形状(batch_size,2)的logits，表示二分类的未归一化分数
    '''
    def forward(self, X):
        # X的形状：(batchsize,num_hiddens)
        return self.output(X)

我们可以看到，`NextSentencePred`实例的前向推断返回每个BERT输入序列的二分类预测。


In [10]:
'''
1. 展平编码器输出
NextSentencePred的输入是整个序列的拼接，模型需要学习从所有词元中判断句间关系
encoded_X原始形状：(batch_size,seq_len,num_hiddens)，例如(2,8,768)
    torch.flatten(...,start_dim=1)：从第1维开始展平
    第0维（batch）保持不变，第1维及以后合并为一个维度
结果形状：(2,8*768)=(2,6144)→每个样本变成一维向量
'''
encoded_X = torch.flatten(encoded_X, start_dim=1)
# NSP的输入形状:(batchsize，num_hiddens)
'''
2. 创建NSP预测头
encoded_X.shape[-1]展平后=6144（8*768）
NextSentencePred(6144)：创建线性层nn.Linear(6144,2)
'''
nsp = NextSentencePred(encoded_X.shape[-1])
'''
3. 执行NSP预测
输入：encoded_X形状(2,6144)
输出：nsp_Y_hat形状(2,2)→每个样本的二分类logits
'''
nsp_Y_hat = nsp(encoded_X)
'''
4. 查看输出形状
2：批量大小
2：二分类logits（是/否下一句）
'''
nsp_Y_hat.shape

torch.Size([2, 2])

还可以计算两个二元分类的交叉熵损失。


**形状匹配详解**
```Python
nsp_Y_hat: 形状 (2, 2)  # 2个样本，2个类别的logits
# 例如: tensor([[-0.34,  1.25],   # 样本0: 不是下一句的分数=-0.34, 是下一句的分数=1.25
#               [ 2.10, -0.78]])  # 样本1: 不是下一句的分数=2.10, 是下一句的分数=-0.78

nsp_y:       形状 (2,)    # 2个样本的真实标签
# 例如: tensor([0, 1])

# CrossEntropyLoss 会自动:
# 1. 对 nsp_Y_hat 做 softmax → 概率
# 2. 根据 nsp_y 选取对应类别的概率
# 3. 计算 -log(prob) 作为损失
```
**计算过程（手动模拟）**
```Python
# 假设 nsp_Y_hat = [[-0.34, 1.25], [2.10, -0.78]]
# 样本0: 真实标签=0 (不是下一句)
#   softmax([-0.34, 1.25]) = [0.20, 0.80]
#   loss0 = -log(0.20) ≈ 1.61

# 样本1: 真实标签=1 (是下一句)
#   softmax([2.10, -0.78]) = [0.95, 0.05]
#   loss1 = -log(0.05) ≈ 2.99

# 如果 reduction='none': nsp_l = [1.61, 2.99]
# 如果 reduction='mean' (默认): nsp_l = (1.61 + 2.99) / 2 = 2.30
```

**输出取决于 loss 的 reduction 参数**：

**情况1：reduction='none'**
```Python
loss = nn.CrossEntropyLoss(reduction='none')
nsp_l = loss(nsp_Y_hat, nsp_y)
print(nsp_l.shape)  # → torch.Size([2])
# 每个元素对应一个样本的损失
# tensor([1.61, 2.99])
```
**情况2：reduction='mean'（默认）**
```Python
loss = nn.CrossEntropyLoss()  # reduction='mean'
nsp_l = loss(nsp_Y_hat, nsp_y)
print(nsp_l.shape)  # → torch.Size([])  (标量)
# tensor(2.30)
```
**情况3：reduction='sum'**
```Python
loss = nn.CrossEntropyLoss(reduction='sum')
nsp_l = loss(nsp_Y_hat, nsp_y)
print(nsp_l.shape)  # → torch.Size([])  (标量)
# tensor(4.60)
```

| 代码行                                | 功能     | 输入形状               | 输出形状 (reduction='mean') | 输出形状 (reduction='none') |
| ---------------------------------- | ------ | ------------------ | ----------------------- | ----------------------- |
| **`nsp_y = torch.tensor([0, 1])`** | 真实标签   | -                  | `(2,)` (long)           | `(2,)` (long)           |
| **`loss = CrossEntropyLoss()`**    | 定义损失函数 | -                  | -                       | -                       |
| **`loss(nsp_Y_hat, nsp_y)`**       | 计算损失   | `(2, 2)` vs `(2,)` | `torch.Size([])` (标量)   | `torch.Size([2])`       |
| **`nsp_l.shape`**                  | 查看损失形状 | -                  | `torch.Size([])`        | `torch.Size([2])`       |


In [11]:
'''
1. 定义真实标签
含义：NSP任务的真实标签，形状(2,)，表示2个样本的标签
标签解释：
    0：第一个样本，句子B不是句子A的下一句（负例）
    1：第二个样本，句子B是句子A的下一句（正例）
'''
nsp_y = torch.tensor([0, 1])
# 2. 计算交叉熵损失
nsp_l = loss(nsp_Y_hat, nsp_y)
#3. 查看损失形状
nsp_l.shape

torch.Size([2])

## 整合代码

In [12]:
'''
核心架构参数
    vocab_size：词汇表大小（如30000）
    num_hiddens：隐藏层维度（768），即词向量维度
    num_heads：多头注意力头数（12）
    num_layers：Transformer层数（12）
    dropout：Dropout比率（0.1）
前馈网络参数
    ffn_num_input：前馈网络输入维度（768）
    ffn_num_hiddens：前馈网络隐藏层维度（3072）
输入维度参数（灵活设计）
    hid_in_features=768：进入self.hidden前的输入维度
    mlm_in_features=768：MLM头的输入维度
    nsp_in_features=768：NSP头的输入维度
    设计目的：允许不同组件使用不同输入维度（虽然默认值相同）
'''
#@save
class BERTModel(nn.Module):
    """BERT模型"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768,
                 nsp_in_features=768):
        super(BERTModel, self).__init__()
        '''
        1. BERT编码器
        功能：将词元ID转换为上下文相关的向量表示
        输出：形状(batch_size,seq_len,num_hiddens)
        '''
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
                    ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                    dropout, max_len=max_len, key_size=key_size,
                    query_size=query_size, value_size=value_size)
        '''
        2. NSP隐藏层（特殊设计）
        作用：对<cls>词元的表示进行非线性变换
        为什么需要：BERT论文中，NSP任务前对<cls>使用Tanh激活，这是特定设计选择
        输入：encoded_X[:,0,:]（<cls>表示，形状(batch,768)）
        输出：形状(batch,num_hiddens)（保持768维）
        '''
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        # 3. MLM预测头
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        '''
        4. NSP预测头
        输入：self.hidden(...)处理后的<cls>表示
        输出：形状(batch,2)（二分类logits）
        '''
        self.nsp = NextSentencePred(nsp_in_features)
    '''
    tokens：词元ID，形状(batch,seq_len)
    segments：片段标记，形状(batch,seq_len)
    valid_lens：有效长度（可选）
    pred_positions：MLM预测位置（可选）
    '''
    def forward(self, tokens, segments, valid_lens=None,
                pred_positions=None):
        # 1. 编码器编码
        encoded_X = self.encoder(tokens, segments, valid_lens) # encoded_X:(batch,seq_len,768)
        '''
        2. MLM任务（条件执行）
        条件判断：如果提供了预测位置，才执行MLM
        设计目的：
            训练时：pred_positions有值，计算MLM损失
            推理/微调时：pred_positions=None，跳过MLM（节省时间）
        '''
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        '''
        3. NSP任务（总是执行）
        encoded_X[:,0,:]：提取<cls>词元的表示
            索引0：<cls>是序列第一个词元
            形状：(batch,768)
        self.hidden(...)：通过Tanh激活层
            形状：(batch,768)
        self.nsp(...)：NSP二分类
            输出：(batch,2)
        '''
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        '''
        4. 返回结果
        encoded_X：完整编码表示（用于下游任务）
        mlm_Y_hat：MLM预测（训练时用）
        nsp_Y_hat：NSP预测（训练时用）
        '''
        return encoded_X, mlm_Y_hat, nsp_Y_hat